In [1]:
import numpy as np
from pymatgen import Molecule
from pymatgen.io.qchem import QcTask, QcOutput
import math

# Get dihedral angle and energy from qchem files

In [2]:
import os 
import re

In [3]:
# this is not placed in utils yet because error checking hasn't been finalized
def get_energy_dihedral(directory):
    dihedral, energy = [], []
    for f in os.listdir(directory):
        if ".qcout" in f:
            if ".orig" not in f:
                output = QcOutput('{d}/{f}'.format(d=directory,f=f))
                qchem_in = output.data[-1]['input']
                try:
                    energy.append(output.final_energy)
                except IndexError:
                    energy.append('no energy') 
                constraints = qchem_in.params['opt']
                for l in constraints:
                    if 'tors' in l:
                        dihedral.append(l[len(l)-1])
    return dihedral, energy

In [4]:
dihedral, energy = get_energy_dihedral('./pt_cust_dft_full')

# Fit and plot dihedral potential

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
from scipy.optimize import curve_fit

In [7]:
# Ryckaert_Bellemans dihedral potential function
def RB_potential(x, a, b, c, d, e, f):
    return (a*1.0 + b*np.cos(x*np.pi/180.0) 
            + c*(np.cos(x*np.pi/180.0)**2) 
            + d*(np.cos(x*np.pi/180.0)**3) 
            + e*(np.cos(x*np.pi/180.0)**4) 
            + f*(np.cos(x*np.pi/180.0)**5))

In [8]:
import utils

In [9]:
rel_eV_energy = utils.relative_energy(energy)

In [10]:
fit_params, fit_covar = curve_fit(RB_potential,dihedral,rel_eV_energy)

In [31]:
# create list of angles and corresponding energies
angles = np.linspace(-180, 180, 3600)
RB_energy = [RB_potential(angle, fit_params[0], fit_params[1], 
                          fit_params[2], fit_params[3], fit_params[4], 
                          fit_params[5]) for angle in angles]

In [30]:
RB_energy1= [RB_potential(angle, *fit_params) for angle in angles]

In [36]:
RB_energy == RB_energy1

True

In [33]:
RB_energy

[0.0052399079270927544,
 0.0052397549212286698,
 0.0052392959206761505,
 0.0052385309765534058,
 0.0052374601740542972,
 0.0052360836324437717,
 0.0052344015050512878,
 0.0052324139792625669,
 0.0052301212765092303,
 0.0052275236522568358,
 0.0052246213959909037,
 0.005221414831201106,
 0.0052179043153636872,
 0.005214090239921954,
 0.0052099730302648832,
 0.0052055531457039003,
 0.0052008310794481255,
 0.0051958073585769383,
 0.0051904825440117009,
 0.0051848572304849516,
 0.0051789320465080143,
 0.0051727076543367544,
 0.0051661847499354123,
 0.005159364062938588,
 0.005152246356611572,
 0.0051448324278085223,
 0.0051371231069292896,
 0.0051291192578738652,
 0.005120821777995126,
 0.0051122315980502769,
 0.0051033496821498029,
 0.0050941770277048819,
 0.0050847146653729474,
 0.0050749636590015686,
 0.0050649251055701165,
 0.0050546001351302232,
 0.0050439899107438957,
 0.0050330956284200543,
 0.0050219185170492283,
 0.0050104598383365236,
 0.0049987208867327071,
 0.004986702989363447

In [ ]:
plt.figure()
plt.plot(dihedral, rel_eV_energy, 'o', angles, RB_energy, 'black')
plt.xlabel('Dihedral angle')
plt.ylabel('Relative eV')
plt.show()

In [ ]:
fit_params

# Boltzmann distribution

In [ ]:
# kbT in eV/KS
kb_eV_K = 8.6173324 * 10**-5

In [ ]:
kbT700 = kb_eV_K * 700.0
kbT300 = kb_eV_K * 300.0

In [ ]:
# normalization 
boltz_factor_700 = [np.exp(-energy / kbT700) for energy in RB_energy]
normalize_val = sum(boltz_factor_700)

In [ ]:
prob_700 = [(np.exp(-energy / kbT700) / normalize_val) for energy in RB_energy]

In [ ]:
plt.figure()
plt.plot(angles, prob_700)
plt.xlabel('Diheadral angle')
plt.ylabel('Probability')
plt.show()

# Map dihedral angles to random numbers between 0-1

In [ ]:
cum_prob = [sum(prob_700[0:prob_i]) for prob_i in range(len(prob_700))]

In [ ]:
plt.figure()
plt.plot(cum_prob, angles)
plt.xlabel('Cumulative probability')
plt.ylabel('Dihedral angle')
plt.show()

In [ ]:
n_prob_angle = np.array(zip(cum_prob, angles))
n_prob_angle

In [ ]:
random = np.random.uniform(0.0, 1.0, size=(24-1))
angle_map = n_prob_angle[:, 0].searchsorted(random)
angle_map = np.array([0, 34, 85])

In [ ]:
dihedral_set = []
for i in angle_map:
    rand = np.random.uniform(0.0, 1.0)
    if rand < 0.5:
        dihedral_set.append(n_prob_angle[i - 1][1])
    else:
        dihedral_set.append(n_prob_angle[i][1])

In [ ]:
n_prob_angle[0][1]

In [ ]:
dihedral_set

# Build, relax, and sample neutral chain

In [ ]:
from polymer_chain import Polymer

In [ ]:
monomer_num = 24
monomer_len = 2.548
link_len = 1.480
link_angle = 15.0
sample_num = 100

In [ ]:
pt = Polymer(monomer_num, monomer_len, link_len, link_angle, n_prob_angle, sample_num)

In [ ]:
#pt_neutal.build_chain()
#pt_neutal.relax_neutral_chain()

In [ ]:
#%time ave_ete, ave_corr = pt.sample_neutral_chains()

In [ ]:
#plt.figure()
#plt.plot(range(0, monomer_num + 1, 1), ave_ete)
#plt.xlabel('number of monomers')
#plt.ylabel('average end_to_end distance')
#plt.show()

In [ ]:
#%timeit pt_neutal.sample_neutral_chains()

In [ ]:
#1 loop, best of 3: 667 ms per loop

In [ ]:
#ave_ete

In [ ]:
#plt.figure()
#plt.plot(range(0, monomer_num + 1, 1), ((ave_ete * ave_ete)*0.01))
#plt.xlabel('number of monomers')
#plt.ylabel('average end_to_end distance')
#plt.show()

In [ ]:
#plt.figure()
#plt.plot(range(0, monomer_num, 1), ave_corr)
#plt.xlabel('number of monomers')
#plt.ylabel('correlation')
#plt.show()

In [ ]:
#plt.figure()
#plt.plot(range(0, monomer_num, 1), np.log(ave_corr))
#plt.xlabel('number of monomers')
#plt.ylabel('correlation')
#plt.show()

# Plot and fit charged dihedral potential

In [ ]:
def get_energy_charged_dihedral(directory):
    dihedral, energy = [], []
    for f in os.listdir(directory):
        #print f
        if ".qcout" in f:
            if '_-90.0' not in f and '_100.0' not in f and '_140' not in f:
                #print f
                output = QcOutput('{d}/{f}'.format(d=directory,f=f))
                qchem_in = output.data[-1]['input']
                try:
                    energy.append(output.final_energy)
                except IndexError:
                    energy.append('no energy') 
                constraints = qchem_in.params['opt']
                for l in constraints:
                    if 'tors' in l:
                        dihedral.append(l[len(l)-1])
    return dihedral, energy

In [ ]:
c_dihedral, c_energy = get_energy_charged_dihedral('./charged_dft_b3lyp_out/')

In [ ]:
cc_energy = []
cc_dihedral = []
for i in range(len(c_energy)):
    if isinstance(c_energy[i], float):
        cc_energy.append(c_energy[i])
        cc_dihedral.append(c_dihedral[i])

In [ ]:
c_rel_eV_energy = utils.relative_energy(cc_energy)

In [ ]:
c_fit_params, c_fit_covar = curve_fit(RB_potential,cc_dihedral,c_rel_eV_energy)

In [ ]:
# create list of angles and corresponding energies
c_angles = np.linspace(-180, 180, 3600)
c_RB_energy = [RB_potential(c_angle, c_fit_params[0], c_fit_params[1], 
                          c_fit_params[2], c_fit_params[3], c_fit_params[4], 
                          c_fit_params[5]) for c_angle in c_angles]

In [ ]:
plt.figure()
plt.plot(cc_dihedral, c_rel_eV_energy, 'o', c_angles, c_RB_energy, 'black')
plt.xlabel('Dihedral angle')
plt.ylabel('Relative eV')
plt.show()

# Boltzmann distribution

In [ ]:
# normalization 
c_boltz_factor_700 = [np.exp(-energy / kbT700) for energy in c_RB_energy]
c_normalize_val = sum(c_boltz_factor_700)

In [ ]:
#c_boltz_factor_700

In [ ]:
prob_700 = [(np.exp(-energy / kbT700) / c_normalize_val) for energy in c_RB_energy]

In [ ]:
plt.figure()
plt.plot(c_angles, prob_700)
plt.xlabel('Diheadral angle')
plt.ylabel('Probability')
plt.show()

# Cumulative Probability

In [ ]:
c_cum_prob = [sum(prob_700[0:prob_i]) for prob_i in range(len(prob_700))]

In [ ]:
plt.figure()
plt.plot(c_cum_prob, c_angles)
plt.xlabel('Cumulative probability')
plt.ylabel('Dihedral angle')
plt.show()

In [ ]:
c_prob_angle = np.array(zip(c_cum_prob, c_angles))

In [ ]:
pt = Polymer(monomer_num, monomer_len, link_len, link_angle, n_prob_angle, sample_num, c_prob_angle)

In [ ]:
pt.build_chain() 
n_chain, n_ete, c_chain, c_ete = pt.relax_charged_chain(1, values=True)

In [ ]:
# [-165.09301860372074]
10

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
ax.plot(n_chain[:,0], n_chain[:,1], n_chain[:,2])

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
ax.plot(c_chain[:,0], c_chain[:,1], c_chain[:,2])

In [ ]:
c_ete, c_corr = pt.sample_charged_chains(1)

In [ ]:
c_ete

In [ ]:
c_corr

In [ ]:
pt.n_random_angle()

In [ ]:
# The slowest run took 5.79 times longer than the fastest. This could mean that an intermediate result is being cached.
#10000 loops, best of 3: 24.7 µs per loop

In [ ]:
#   try:
                    #dihedral_set.append(self.n_prob_angle[prob_i][1])
                #except IndexError:

In [ ]:
n_prob_angle[3600][1]

In [ ]:
n_prob_angle[3599][1]